In [1]:
import numpy as np
from scipy import stats
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from matplotlib.transforms import Bbox
import plotly.graph_objects as go
import pickle as pkl
import keras
import os
from GraphFuncsV3 import*
from matplotlib import cm

In [2]:
model_id = 'ConFormer_1.0e-03_15_1_3'
modelname = 'realdata_%s' % model_id
model = keras.models.load_model('./results/%s.h5' % model_id)

#Create folders for plots
try:
    os.mkdir("./Plots/%s" % modelname)
    os.mkdir("./Plots/%s/Model" % modelname)
    os.mkdir("./Plots/%s/Model/Single Model" % modelname)
    os.mkdir("./Plots/%s/Model(3)" % modelname)
    os.mkdir("./Plots/%s/Model(3)/Single Model" % modelname)
    os.mkdir("./Plots/%s/Model(5)" % modelname)
    os.mkdir("./Plots/%s/Model(5)/Single Model" % modelname)
    os.mkdir("./Plots/%s/Comparison" % modelname)
    os.mkdir("./Plots/%s/Comparison/Single Model" % modelname)
    os.mkdir("./Plots/%s/Analytical" % modelname)
    os.mkdir("./Plots/%s/Analytical/Single Model" % modelname)
except OSError:
    print("Folder %s already exists" % modelname)

Folder realdata_ConFormer_1.0e-03_15_1_3 already exists


In [3]:
savedir = "./results/"
yscaler = pkl.load(open(savedir + "yscaler", "rb"))
testin = pkl.load(open(savedir + "realdatain", "rb"))
testout = pkl.load(open(savedir + "realdataout", "rb"))
testout = testout.numpy()

In [4]:
true = testout[:,5]
pred = model.predict(testin)
pred = yscaler.inverse_transform(pred)[:,0]
chisquare_pred = pkl.load(open("./results_single/single_parameters.pkl", "rb"))
chisquare_pred = np.array(chisquare_pred)[:,5]
chisquare_pred = np.delete(chisquare_pred, 257, 0)

In [5]:
print(np.shape(pred))
print(np.shape(true))
print(np.shape(chisquare_pred))

(363,)
(363,)
(363,)


In [6]:
varname = [
#            "Amplitude", 
#            "Decay Coefficient", 
#            "Spherical Aberration", 
#            "Astigmatism", 
#            "Principal Axis Angle", 
           "Defocus", 
#            "Detuning"
          ]
bins = 10

# for i in range(len(varname)):
#     lower, upper = getlowerupper(pred[:], true[:])
# #     lower = -0.5
# #     upper = 1.8
#     histo(modelname, true[:], pred[:], varname[i], lower, upper, bins)
#     sems, resmeans, lowerranges, upperranges, lowerbounds, upperbounds, samples = \
#     plotgaussian(modelname, true[:], pred[:], varname[i], lower, upper, bins)
#     rmsplot(modelname, true[:], pred[:], varname[i], lower, upper, bins)
#     heatmap(modelname, true[:], pred[:], varname[i], lower, upper, bins)
#     confinttable(modelname, varname[i], sems, resmeans, lowerranges, upperranges, lowerbounds, upperbounds, samples)

In [7]:
bins = 10
space = 'Single Model'

for i in range(len(varname)):
    lower, upper = getlowerupper(pred[:], true[:])
    
    preds = np.array([pred[:], chisquare_pred[:]])
#     preds = np.array([pred[:]])
    
    prednames = ['ML Model', 'Analytical']
#     prednames = ['ML Model']
    
    histo(modelname, 
          'Comparison', 
          space, 
          true[:], 
          preds, 
          varname[i], 
          prednames, 
          lower, 
          upper, 
          bins)
    
    RMSEcompare(modelname, 
                space, 
                true[:], 
                preds, 
                varname[i], 
                prednames, 
                lower, 
                upper, 
                bins)
    
    heatmap(modelname, 
            'Model', 
            space, 
            true[:], 
            pred[:], 
            varname[i], 
            lower, 
            upper, 
            bins)
    
    heatmap(modelname, 
            'Analytical', 
            space, 
            true[:],
            chisquare_pred[:], 
            varname[i], 
            lower, 
            upper, 
            bins)
    
    resmeans, stddevs, bincenters, samples, resmean, std = \
    plotgaussian(modelname, 
                 'Model', 
                 space, 
                 true[:], 
                 pred[:], 
                 varname[i], 
                 lower, 
                 upper, 
                 bins)
            
    chisquare_resmeans, chisquare_stddevs, chisquare_bincenters, chisquare_samples, chisquare_resmean, chisquare_std = \
    plotgaussian(modelname, 
                 'Analytical', 
                 space, 
                 true[:], 
                 chisquare_pred[:], 
                 varname[i], 
                 lower, 
                 upper, 
                 bins)
    
    scatter(modelname,
            space,
            [resmeans, chisquare_resmeans], 
            [bincenters, chisquare_bincenters], 
            [samples, chisquare_samples], 
            varname[i], 
            ['ML Model', 'Analytical'], 
            'Mean')
    
    
    scatter(modelname,
            space,
            [stddevs, chisquare_stddevs], 
            [bincenters, chisquare_bincenters], 
            [samples, chisquare_samples], 
            varname[i], 
            ['ML Model', 'Analytical'],
            'Stddev')
    
    
    confinttable(modelname, 
                 space, 
                 'Model', 
                 resmean, 
                 std, 
                 varname[i], 
                 sum(samples), 
                 lower, 
                 upper)
    
    confinttable(modelname, 
                 space, 
                 'Analytical', 
                 chisquare_resmean, 
                 chisquare_std, 
                 varname[i], 
                 sum(chisquare_samples), 
                 lower, 
                 upper)

C:\Users\eric0\Jupyter Notebooks\Cold Physics\GraphFuncsV3.py:96: RuntimeWarning: divide by zero encountered in true_divide
  binratio = bincountsx / bincountsy
C:\Users\eric0\Jupyter Notebooks\Cold Physics\GraphFuncsV3.py:405: RuntimeWarning: invalid value encountered in true_divide
  axs[1].scatter(xcenters, binstd1 / binstd2)
C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\core\_methods.py:194: RuntimeWarning: invalid value encountered in t

In [8]:
threeimages = pkl.load(open(savedir + "3images", "rb"))
threeimparams = pkl.load(open(savedir + "3imparams", "rb"))
threeimparams = threeimparams.numpy()
true = threeimparams.copy()
pred3 = model.predict(threeimages)
pred3 = yscaler.inverse_transform(pred3)[:,0]

In [9]:
print(np.shape(true))

(111,)


In [10]:
bins = 10
space = 'Single Model'

for i in range(len(varname)):
    lower, upper = getlowerupper(pred3[:], true[:])
    
    prednames = ['ML Model(3)', 'Analytical']
#     prednames = ['ML Model']
    
    histo(modelname, 
          'Comparison', 
          space, 
          true[:], 
          preds, 
          varname[i], 
          prednames, 
          lower, 
          upper, 
          bins)
    
    heatmap(modelname, 
            'Model(3)', 
            space, 
            true[:], 
            pred3[:], 
            varname[i], 
            lower, 
            upper, 
            bins)   
    
    resmeans, stddevs, bincenters, samples, resmean, std = \
    plotgaussian(modelname, 
                 'Model(3)', 
                 space, 
                 true[:], 
                 pred3[:], 
                 varname[i], 
                 lower, 
                 upper, 
                 bins)    
    
    confinttable(modelname, 
                 space, 
                 'Model(3)', 
                 resmean, 
                 std, 
                 varname[i], 
                 sum(samples), 
                 lower, 
                 upper)

C:\Users\eric0\Jupyter Notebooks\Cold Physics\GraphFuncsV3.py:96: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning:

Mean of empty slice.

C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning:

Degrees of freedom <= 0 for slice

C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\core\_methods.py:194: RuntimeWarning:

invalid value encountered in true_divide

C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\lib\histograms.py:905: RuntimeWarning:

invalid value encountered in true_divide

C:\Users\eric0\.conda\envs\Phys590\lib\site-packag

In [11]:
fiveimages = pkl.load(open(savedir + "5images", "rb"))
fiveimparams = pkl.load(open(savedir + "5imparams", "rb"))
fiveimparams = fiveimparams.numpy()
true = fiveimparams.copy()
pred5 = model.predict(fiveimages)
pred5 = yscaler.inverse_transform(pred5)[:,0]

In [12]:
bins = 10
space = 'Single Model'

for i in range(len(varname)):
    lower, upper = getlowerupper(pred5[:], true[:])
    
    prednames = ['ML Model(5)', 'Analytical']
#     prednames = ['ML Model']
    
    histo(modelname, 
          'Comparison', 
          space, 
          true[:], 
          preds, 
          varname[i], 
          prednames, 
          lower, 
          upper, 
          bins)
    
    heatmap(modelname, 
            'Model(5)', 
            space, 
            true[:], 
            pred5[:], 
            varname[i], 
            lower, 
            upper, 
            bins)   
    
    resmeans, stddevs, bincenters, samples, resmean, std = \
    plotgaussian(modelname, 
                 'Model(5)', 
                 space, 
                 true[:], 
                 pred5[:], 
                 varname[i], 
                 lower, 
                 upper, 
                 bins)    
    
    confinttable(modelname, 
                 space, 
                 'Model(5)', 
                 resmean, 
                 std, 
                 varname[i], 
                 sum(samples), 
                 lower, 
                 upper)

C:\Users\eric0\Jupyter Notebooks\Cold Physics\GraphFuncsV3.py:96: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning:

Mean of empty slice.

C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning:

Degrees of freedom <= 0 for slice

C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\core\_methods.py:194: RuntimeWarning:

invalid value encountered in true_divide

C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\eric0\.conda\envs\Phys590\lib\site-packages\numpy\lib\histograms.py:905: RuntimeWarning:

invalid value encountered in true_divide

C:\Users\eric0\.conda\envs\Phys590\lib\site-packag

In [13]:
imgSysData = { 
    "CCDPixelSize": 13,      # pixel size of the CCD, in micron 
    "magnification": 27,      # 799.943 / 29.9099, # magnification of the imaging system 
    "wavelen"     : 0.852,     # wavelength of the imaging beam, in micron 
    "NA"          : 0.37,      # numerical aperture of the objective 
    "ODtoAtom"    : 13
}

K_x, K_y = pkl.load(open(savedir + "K_xK_y", "rb"))

from NPSmethods2 import pupilFunc, getFreq

#Generate M2k_Fit
def make_M2k_Fit(paras, imgSysData):
    
    A, tau, S0, alpha, phi, beta, delta_s = paras
    _, _, K_X, K_Y = getFreq(imgSysData["CCDPixelSize"],
                                      imgSysData["magnification"], (100,100))
    d = imgSysData["wavelen"] / (2*np.pi*imgSysData["NA"])
    R_p, Theta_p = np.abs(K_X + 1j*K_Y) * d, np.angle(K_X + 1j*K_Y)
    p1 = pupilFunc(R_p, Theta_p + np.pi, tau, S0, alpha, phi, beta)
    p2 = np.conj(pupilFunc(R_p, Theta_p, tau, S0, alpha, phi, beta)) * \
            np.exp(-2*1j*delta_s)
    PSF = (p1 + p2) / (2 * np.cos(delta_s))
    M2k = np.abs(PSF)**2
    M2k_Fit = A * M2k
    M2k_Fit[M2k_Fit.shape[0]//2, M2k_Fit.shape[1]//2] = 0
    
    return M2k_Fit

try:
    os.mkdir("./results/Predicted Images ML/")
except OSError:
    print("Folder Predicted Images ML already exists")
    
try:
    os.mkdir("./results/Predicted Images Analytical/")
except OSError:
    print("Folder Predicted Images Analytical already exists")

Folder Predicted Images ML already exists
Folder Predicted Images Analytical already exists


In [14]:
print(testout[0])
print(testout[50])

[ 0.33071131  1.          0.01        2.46       -1.9268653   0.36400178
 -0.03788753]
[ 2.15445973e-01  1.00000000e+00  2.23904917e-22  2.46000000e+00
 -1.88572251e+00 -2.00000000e+00 -1.05513881e-01]


In [15]:
for paranum in range(len(testout)):
    testout[paranum][5]  = pred[paranum]
    M2k_Fit = make_M2k_Fit(testout[paranum], imgSysData)
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, M2k_Fit, cmap=cm.jet, vmin=0, vmax=M2k_Fit.max())
    plt.colorbar(im)
    fig.savefig(savedir + 'Predicted Images ML/Single' + str(paranum + 1))
    plt.close(fig)
    
for paranum in range(len(testout)):
    testout[paranum][5]  = chisquare_pred[paranum]
    M2k_Fit = make_M2k_Fit(testout[paranum], imgSysData)
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, M2k_Fit, cmap=cm.jet, vmin=0, vmax=M2k_Fit.max())
    plt.colorbar(im)
    fig.savefig(savedir + 'Predicted Images Analytical/Single' + str(paranum + 1))
    plt.close(fig)

# for paranum in range(len(threeimparams)):
#     threeimparams[paranum][5] = pred3[paranum]
#     M2k_Fit = make_M2k_Fit(threeimparams[paranum], imgSysData)
#     fig = plt.figure(figsize=(5.5, 4.5))
#     ax = fig.add_subplot(111)
#     im = ax.pcolor(K_x, K_y, M2k_Fit, cmap=cm.jet, vmin=0, vmax=M2k_Fit.max())
#     plt.colorbar(im)
#     fig.savefig(savedir + 'Predicted Images/Three' + str(paranum + 1))
#     plt.close(fig)
    
# for paranum in range(len(fiveimparams)):
#     fiveimparams[paranum][5] = pred5[paranum]
#     M2k_Fit = make_M2k_Fit(fiveimparams[paranum], imgSysData)
#     fig = plt.figure(figsize=(5.5, 4.5))
#     ax = fig.add_subplot(111)
#     im = ax.pcolor(K_x, K_y, M2k_Fit, cmap=cm.jet, vmin=0, vmax=M2k_Fit.max())
#     plt.colorbar(im)
#     fig.savefig(savedir + 'Predicted Images/Five' + str(paranum + 1))
#     plt.close(fig)

<ipython-input-15-a7753f1b1cba>:6: MatplotlibDeprecationWarning:

shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.

<ipython-input-15-a7753f1b1cba>:16: MatplotlibDeprecationWarning:

shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.

